In [1]:
import os
from PIL import Image
from tqdm import tqdm
import tensorflow_datasets as tfds
import tensorflow as tf
from pathlib import Path
import numpy as np
import re

# ==== 参数设置 ====
builder_dir = r"E:\bridgev2\0.1.0"      # TFDS 数据目录（含 dataset_info.json / features.json / tfrecord-*）
# split       = "train"
# frame_indices = [0]                 # [0]=第一帧；[-1]=最后一帧；[0, -1]=第一+最后；None=全部帧



In [2]:
# 使用 builder_from_directory 加载数据集
builder = tfds.builder_from_directory(builder_dir)

# 获取数据集的 info 信息
info = builder.info

# 打印数据集的基本信息
print("Dataset Name:", builder.name)
print("Dataset Version:", info.version)
print("Dataset Homepage:", info.homepage)
print("Dataset Description:", info.description)

# 打印 splits 信息
print("\nSplits Information:")
for split, split_info in info.splits.items():
    print(f"{split}: {split_info.num_examples} examples, {split_info.num_bytes:,} bytes")

# 打印 features 信息
print("\nInfo:")
print(info)

# 打印监督的键
print("\nSupervised Keys:")
print(info.supervised_keys)


Dataset Name: bridge
Dataset Version: 0.1.0
Dataset Homepage: https://rail-berkeley.github.io/bridgedata/
Dataset Description: WidowX interacting with toy kitchens

Splits Information:
train: 25460 examples, 365,981,955,835 bytes
test: 3475 examples, 50,084,717,377 bytes

Info:
tfds.core.DatasetInfo(
    name='bridge',
    full_name='bridge/0.1.0',
    description="""
    WidowX interacting with toy kitchens
    """,
    homepage='https://rail-berkeley.github.io/bridgedata/',
    data_dir='E:\\bridgev2\\0.1.0',
    file_format=tfrecord,
    download_size=Unknown size,
    dataset_size=387.49 GiB,
    features=FeaturesDict({
        'steps': Dataset({
            'action': FeaturesDict({
                'open_gripper': bool,
                'rotation_delta': Tensor(shape=(3,), dtype=float32),
                'terminate_episode': float32,
                'world_vector': Tensor(shape=(3,), dtype=float32),
            }),
            'is_first': bool,
            'is_last': bool,
         

In [3]:
ds = builder.as_dataset(split='test')

In [4]:
max_to_check = 100000  # 防止太慢，可设大一点
count = 0

# 这里不再使用 ds[split]，因为 ds 已经是一个 Dataset 了
for _ in tqdm(ds.take(max_to_check), desc="Counting samples"):
    break

Counting samples:   0%|          | 0/3475 [00:08<?, ?it/s]


## 分别提取

In [ ]:
# ==== 输出准备 ====
output_path  = r"F:\PangYe\bridgev2_DATA\extracted" # 输出目录
output_dir = Path(output_path)
output_dir.mkdir(parents=True, exist_ok=True)
max_samples = 10                     # None 表示全部样本；设为整数表示只处理这么多

# ==== 迭代样本 ====
pbar = tqdm(enumerate(ds.take(max_samples), start=1), total=max_samples, desc="Extracting samples")

for sample_idx, example in pbar:
    steps = example["steps"]

    # 创建当前 episode 文件夹：id00001, id00002, ...
    ep_dir = output_dir / f"id{sample_idx:05d}"
    images_dir = ep_dir / "images"
    images_dir.mkdir(parents=True, exist_ok=True)

    lang = None

    # 遍历 step 并保存图片
    for step_idx, step in enumerate(steps.as_numpy_iterator(), start=1):
        img = step["observation"]["image"]
        lang_text = step["observation"]["natural_language_instruction"].decode("utf-8")

        # 记录第一步的自然语言指令
        if step_idx == 1:
            lang = lang_text

        # 保存图像到 images/ 目录
        img_pil = Image.fromarray(img)
        img_name = f"step{step_idx:03d}.png"
        img_pil.save(images_dir / img_name)

    # 保存语言到 instruction.txt（与 images 平级）
    if lang is not None:
        with open(ep_dir / "instruction.txt", "w", encoding="utf-8") as f:
            f.write(lang.strip())

pbar.close()
print(f"\n✅ 提取完成，共保存 {min(max_samples, builder.info.splits[split].num_examples)} 个 episode。")

In [ ]:
# ==== 输出准备 ====
output_path  = r"F:\PangYe\bridgev2_DATA\extracted_1" # 输出目录
output_dir = Path(output_path)
output_dir.mkdir(parents=True, exist_ok=True)
max_samples = 10                     # None 表示全部样本；设为整数表示只处理这么多

# ==== 迭代样本 ====
pbar = tqdm(enumerate(ds.take(max_samples), start=1), total=max_samples, desc="Extracting samples")

for sample_idx, example in pbar:
    steps = example["steps"]

    # 创建当前 episode 文件夹：id00001, id00002, ...
    ep_dir = output_dir / f"id{sample_idx:05d}"
    ep_dir.mkdir(parents=True, exist_ok=True)

    # === 1. 记录首步语言指令 ===
    lang = None

    # === 2. 遍历 step 并保存图片 ===
    for step_idx, step in enumerate(steps.as_numpy_iterator(), start=1):
        img = step["observation"]["image"]
        lang_text = step["observation"]["natural_language_instruction"].decode("utf-8")

        # 保存第一个 step 的语言
        if step_idx == 1:
            lang = lang_text

        # 保存图像
        img_pil = Image.fromarray(img)
        img_name = f"step{step_idx:03d}.png"
        img_pil.save(ep_dir / img_name)

    # === 3. 保存语言到 txt 文件 ===
    if lang is not None:
        with open(ep_dir / "instruction.txt", "w", encoding="utf-8") as f:
            f.write(lang.strip())

pbar.close()
print(f"\n✅ 提取完成，共保存 {min(max_samples, builder.info.splits[split].num_examples)} 个 episode。")

## 取九

In [ ]:
# ==== 迭代样本（使用 np.linspace 采样9帧，含首尾） ====
output_path  = r"F:\PangYe\bridgev2_DATA\extracted_nine_train" # 输出目录
output_dir = Path(output_path)
output_dir.mkdir(parents=True, exist_ok=True)
max_samples = 25460                     # None 表示全部样本；设为整数表示只处理这么多

pbar = tqdm(enumerate(ds.take(max_samples), start=1), total=max_samples, desc="Extracting 9 frames (linspace)")


for sample_idx, example in pbar:
    steps = list(example["steps"].as_numpy_iterator())
    num_steps = len(steps)

    ep_dir = output_dir / f"id{sample_idx:05d}"
    images_dir = ep_dir / "images"
    images_dir.mkdir(parents=True, exist_ok=True)

    # === 1. 提取第一步语言指令 ===
    lang = steps[0]["observation"]["natural_language_instruction"].decode("utf-8")

    # === 2. 计算9个等间隔索引（包含首帧与末帧） ===
    if num_steps <= 9:
        frame_indices = np.arange(num_steps)
    else:
        frame_indices = np.linspace(0, num_steps - 1, num=9, dtype=int)

    # === 3. 保存这些帧 ===
    for local_idx, step_idx in enumerate(frame_indices, start=1):
        img = steps[step_idx]["observation"]["image"]
        img_pil = Image.fromarray(img)
        img_name = f"step{step_idx:03d}.png"
        img_pil.save(images_dir / img_name)

    # === 4. 保存语言文件 ===
    with open(ep_dir / "instruction.txt", "w", encoding="utf-8") as f:
        f.write(lang.strip())

pbar.close()
print(f"\n✅ 提取完成（每个 episode 使用 np.linspace 等间隔选取 9 帧，含首尾），共保存 {min(max_samples, builder.info.splits[split].num_examples)} 个 episode。")


In [ ]:
# ==== 配置 ====
output_path  = r"F:\PangYe\bridgev2_DATA\extracted_nine_train"
output_dir = Path(output_path)
output_dir.mkdir(parents=True, exist_ok=True)
MAX_SAMPLES = 20000   # None 表示不限制

ID_RE = re.compile(r"^id(\d{5})$")

def is_episode_complete(ep_dir: Path) -> bool:
    """判断该 episode 是否完整（有 instruction.txt 且 images 里 >=9 张）"""
    images_dir = ep_dir / "images"
    if not images_dir.exists():
        return False
    pngs = list(images_dir.glob("*.png"))
    instr_ok = (ep_dir / "instruction.txt").exists()
    return instr_ok and len(pngs) >= 9

# ==== 统计已完成的样本数量（只计算真正完整的）====
completed_ids = []
for d in sorted(output_dir.iterdir()):
    if d.is_dir() and ID_RE.match(d.name) and is_episode_complete(d):
        completed_ids.append(int(d.name[-5:]))

resume_from = len(completed_ids)  # 已完整完成的 episode 数量
print(f"[INFO] 已完成 {resume_from} 个 episode，将从第 {resume_from+1} 个继续。")

# ==== 计算本次要处理的目标数量 ====
if MAX_SAMPLES is None:
    to_take = None
else:
    # 如果之前已经达到/超过 MAX_SAMPLES，就不再处理
    remaining = max(0, MAX_SAMPLES - resume_from)
    if remaining == 0:
        print("[INFO] 已达到 MAX_SAMPLES，无需继续。")
        raise SystemExit
    to_take = remaining

# ==== 基于 skip() 进行续跑 ====
ds_resumed = ds.skip(resume_from)
if to_take is not None:
    ds_resumed = ds_resumed.take(to_take)

pbar = tqdm(enumerate(ds_resumed, start=resume_from + 1),
            total=to_take, desc="Extracting 9 frames (linspace)")

for sample_idx, example in pbar:
    steps = list(example["steps"].as_numpy_iterator())
    num_steps = len(steps)

    ep_dir = output_dir / f"id{sample_idx:05d}"
    images_dir = ep_dir / "images"
    images_dir.mkdir(parents=True, exist_ok=True)

    # 若该 episode 已完整，直接跳过（防止重复写）
    if is_episode_complete(ep_dir):
        pbar.set_postfix_str("skip (already complete)")
        continue

    # === 1. 提取第一步语言指令 ===
    lang = steps[0]["observation"]["natural_language_instruction"].decode("utf-8")

    # === 2. 计算9个等间隔索引（包含首帧与末帧） ===
    if num_steps <= 9:
        frame_indices = np.arange(num_steps)
    else:
        # 用 round 再转 int，分布更均匀；去重以防 round 碰撞
        cand = np.round(np.linspace(0, num_steps - 1, num=9)).astype(int)
        frame_indices = np.unique(cand)

    # === 3. 保存这些帧 ===
    for step_idx in frame_indices:
        img = steps[step_idx]["observa" \
        "" \
        "" \
        "" \
        "tion"]["image"]
        img_pil = Image.fromarray(img)
        img_name = f"step{step_idx:03d}.png"
        img_pil.save(images_dir / img_name)

    # === 4. 保存语言文件 ===
    with open(ep_dir / "instruction.txt", "w", encoding="utf-8") as f:
        f.write(lang.strip())

pbar.close()
print(f"\n✅ 续跑完成：从 id{resume_from+1:05d} 开始处理，共新增 {to_take if to_take is not None else '若干'} 个 episode。")

## 提取视频

In [ ]:
# %%
# ==== 基于 bridge/0.1.0 的视频导出（单相机：observation.image） ====
import cv2, json
from pathlib import Path

# === 可配置项 ===
OUTPUT_ROOT = Path(r"F:\PangYe\bridgev2_DATA\videos_train")  # 导出根目录
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)
FPS = 15                  # 采样帧率（不知道真实帧率时填 15/20/30 都可）
USE_LOSSLESS = True       # 优先无损（FFV1 -> .mkv）
MAX_SAMPLES = 25460       # None 表示不限制
SAMPLE_PAD = 5            # id00001 的位宽
DS_SPLIT = "train"        # 仅用于统计信息显示；ds 已在上文用 train 构造

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p

def decode_str(x) -> str:
    if x is None:
        return ""
    if isinstance(x, bytes):
        return x.decode("utf-8", errors="ignore")
    return str(x)

def _make_writer(out_path: Path, frame_size_hw, fps=FPS, lossless=USE_LOSSLESS):
    """优先 FFV1（.mkv），失败回退 MJPG（.avi）"""
    H, W = map(int, frame_size_hw)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    if lossless:
        try:
            fourcc = cv2.VideoWriter_fourcc(*'FFV1')
            vw = cv2.VideoWriter(str(out_path.with_suffix('.mkv')), fourcc, fps, (W, H))
            if vw.isOpened():
                return vw, '.mkv'
        except Exception:
            pass

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    vw = cv2.VideoWriter(str(out_path.with_suffix('.avi')), fourcc, fps, (W, H))
    if not vw.isOpened():
        raise RuntimeError(f"Cannot open VideoWriter for {out_path}")
    return vw, '.avi'

# === JSONL（只要首步指令非空就写一行，并导出对应视频） ===
jsonl_path = OUTPUT_ROOT / "language_instructions.jsonl"
jsonl_path.parent.mkdir(parents=True, exist_ok=True)
jsonl_f = open(jsonl_path, "a", encoding="utf-8")

total_examples = info.splits[DS_SPLIT].num_examples or None

seen_samples = 0       # 遍历过的样本计数（用于生成连续 sample_id）
wrote_videos = 0       # 实际写出视频的样本计数

pbar = tqdm(total=(MAX_SAMPLES if MAX_SAMPLES is not None else total_examples),
            desc="Bridge videos (only if instruction present)", unit="ep")

for sample_index, example in enumerate(ds, start=1):
    seen_samples += 1
    sample_id = f"id{seen_samples:0{SAMPLE_PAD}d}"

    if MAX_SAMPLES is not None and seen_samples > MAX_SAMPLES:
        pbar.write(f"[INFO] Reached MAX_SAMPLES={MAX_SAMPLES}; stopping.")
        break

    # 没有 steps 的异常样本：跳过（ID 仍占用以保证连续）
    if "steps" not in example:
        pbar.write(f"[WARN] {sample_id}: no 'steps'; skipped.")
        pbar.update(1)
        continue

    steps_ds = example["steps"]
    if not isinstance(steps_ds, tf.data.Dataset):
        pbar.write(f"[WARN] {sample_id}: 'steps' not tf.data.Dataset; skipped.")
        pbar.update(1)
        continue

    # 窥视第一步，读取首步自然语言指令（bridge: observation.natural_language_instruction）
    first_step_np = None
    for _first in steps_ds.take(1):
        first_step_np = tf.nest.map_structure(lambda x: x.numpy(), _first)
    if first_step_np is None:
        pbar.write(f"[WARN] {sample_id}: 0 steps; skipped.")
        pbar.update(1)
        continue

    obs0 = first_step_np.get("observation", {})
    instr = decode_str(obs0.get("natural_language_instruction", b"")).strip()
    has_text = bool(instr)

    if not has_text:
        # 首步文本为空：完全跳过（与“提取视频的方法”一致：只对有文本的样本建视频/写JSONL）
        pbar.set_postfix_str("skip (empty instruction)")
        pbar.update(1)
        continue

    # 写 JSONL（仅一条指令）
    rec = {"sample_id": sample_id, "instruction": instr}
    jsonl_f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    jsonl_f.flush()

    # 准备输出目录与 writer（单相机：observation.image）
    sample_dir = ensure_dir(OUTPUT_ROOT / sample_id)
    out_base = sample_dir / "image"   # 不带扩展名；根据编码自动加 .mkv / .avi

    writer = None
    suffix = None
    frame_size = None
    frames_written = 0

    # 逐步写帧
    for t, step in enumerate(steps_ds):
        step_np = tf.nest.map_structure(lambda x: x.numpy(), step)
        obs = step_np.get("observation", {})
        if "image" not in obs:
            pbar.write(f"[WARN] {sample_id} step {t}: missing observation.image; frame skipped.")
            continue

        rgb = obs["image"]  # (H, W, 3) uint8
        if frame_size is None:
            H, W = int(rgb.shape[0]), int(rgb.shape[1])
            frame_size = (H, W)
            writer, suffix = _make_writer(out_base, frame_size, fps=FPS, lossless=USE_LOSSLESS)

        bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
        writer.write(bgr)
        frames_written += 1

    # 关闭 writer
    if writer is not None:
        writer.release()

    if frames_written == 0:
        pbar.write(f"[WARN] {sample_id}: no valid frames written; video likely empty.")
    else:
        wrote_videos += 1

    pbar.update(1)

pbar.close()
jsonl_f.close()

print(f"Language JSONL saved to: {jsonl_path}")
print(f"Seen samples (IDs assigned): {seen_samples}")
print(f"Episodes with videos written: {wrote_videos}")
print(f"Output root: {OUTPUT_ROOT}")


## 第一帧

In [6]:
# ==== 迭代样本（仅提取第一帧） ====
output_path  = r"E:\bridgev2_DATA\extracted_first_train" # 输出目录
output_dir = Path(output_path)
output_dir.mkdir(parents=True, exist_ok=True)
max_samples = 1000                    # None 表示全部样本；设为整数表示只处理这么多


pbar = tqdm(enumerate(ds.take(max_samples), start=1), total=max_samples, desc="Extracting first frame only")

for sample_idx, example in pbar:
    steps = list(example["steps"].as_numpy_iterator())

    # ep_dir = output_dir / f"id{sample_idx:05d}"
    # images_dir = ep_dir / "images"
    # images_dir.mkdir(parents=True, exist_ok=True)

    # === 1. 第一步的 step ===
    first_step = steps[0]
    img = first_step["observation"]["image"]
    lang = first_step["observation"]["natural_language_instruction"].decode("utf-8")

    # === 2. 保存图像 ===
    img_pil = Image.fromarray(img)
    img_pil.save(output_dir / f"id{sample_idx:05d}_step001.png")

    # # === 3. 保存语言文件 ===
    # with open(ep_dir / "instruction.txt", "w", encoding="utf-8") as f:
    #     f.write(lang.strip())

pbar.close()
print(f"\n✅ 提取完成（每个 episode 仅保存第一帧），共保存 {min(max_samples, builder.info.splits[split].num_examples)} 个 episode。")


Extracting first frame only: 100%|██████████| 1000/1000 [01:06<00:00, 14.97it/s]


✅ 提取完成（每个 episode 仅保存第一帧），共保存 1000 个 episode。


In [5]:
output_path = r"E:\bridgev2_DATA\extracted_first_test_2"  # 输出根目录
output_dir = Path(output_path)
output_dir.mkdir(parents=True, exist_ok=True)

max_samples = 550  # None 表示全部样本；设为整数表示只处理这么多
batch_size  = 1000  # 每1000个样本一个子文件夹

pbar = tqdm(enumerate(ds.take(max_samples), start=1), total=max_samples, desc="Extracting first frame only")

for sample_idx, example in pbar:
    steps = list(example["steps"].as_numpy_iterator())
    first_step = steps[0]
    img = first_step["observation"]["image"]
    lang = first_step["observation"]["natural_language_instruction"].decode("utf-8")

    # === 计算当前 batch 号（从1开始） ===
    batch_id = (sample_idx - 1) // batch_size + 1
    batch_dir = output_dir / f"batch_{batch_id:05d}"
    batch_dir.mkdir(parents=True, exist_ok=True)

    # === 保存图像 ===
    img_pil = Image.fromarray(img)
    img_pil.save(batch_dir / f"id{sample_idx:05d}_step001.png")

    # === 保存语言文件 ===
    # with open(batch_dir / f"id{sample_idx:05d}_instruction.txt", "w", encoding="utf-8") as f:
    #     f.write(lang.strip())

pbar.close()
print(f"\n✅ 提取完成（每个 episode 仅保存第一帧），共保存 {min(max_samples, builder.info.splits[split].num_examples)} 个 episode。")

Extracting first frame only: 100%|██████████| 550/550 [00:41<00:00, 13.32it/s]


✅ 提取完成（每个 episode 仅保存第一帧），共保存 550 个 episode。
